In [1]:
from hmm import HMM
import io_utils

In [2]:
OUTPUT_DIR = 'smoothed_midi_trigram_test'

In [3]:
DIRS = ['debussy', 'mozart', 'beeth', 'chopin']
# buckets are floats with optional params ('d': dotted, 't': triplet)
BUCKETS = [(1.0/16, 'd'), (1.0/16, 't'), (1.0/16,),
            (1.0/8, 'd'), (1.0/8, 't'), (1.0/8,),
            (1.0/4, 'd'), (1.0/4, 't'), (1.0/4,),
            (1.0/2, 'd'), (1.0/2,),
            (1.0,)]

In [4]:
def outputDiffs(H, E, filenames):
    for i in range(len(H)):
        diffs = 0
        total = 0
        for j in range(len(H[i])):
            if H[i][j] != E[i][j]:
                diffs += 1
            total += 1
        if total == 0:
            print filenames[i], 'has no notes'
        else:
            print filenames[i], 'Percent diffs: ', float(diffs) / total * 100   

In [5]:
def possibleHMMBuckets(buckets):
    # returns list of ((bucket, bool)) for all possible combos
    notes = [(bucket, True) for bucket in buckets]
    rests = [(bucket, False) for bucket in buckets]
    return notes+rests

In [6]:
# lists of lists, 1 list per file
# H_mat[0] = [(bucket, note_list, volume)] w/ bucket in BUCKETS
# note_list list of pitches (MIDI standard, volume int
# E_mat[0] same
# tempos is list of median tempos for reconstuction purposes
# signatures [(timeSig, keySig)] midiEvents for each file
H_mat, E_mat, tempos, filenames, signatures = io_utils.generateTrainData(DIRS, BUCKETS,allowed_tempo_diff=10)

[0.375, 0.16666666666666666, 0.25, 0.75, 0.3333333333333333, 0.5, 1.5, 0.6666666666666666, 1.0, 3.0, 2.0, 4.0]
debussy/DEB_CLAI.mid
debussy/deb_prel.mid
debussy/DEB_PASS.mid
ms_per_tick 719.177083333
debussy/debussy_cc_6.mid
ms_per_tick 1126.12708333
debussy/debussy_cc_4.mid
ms_per_tick 1011.98125
debussy/deb_menu.mid
ms_per_tick 1678.07708333
debussy/debussy_cc_1.mid
debussy/debussy_cc_2.mid
debussy/debussy_cc_3.mid
ms_per_tick 920.675
mozart/mz_570_1.mid
ms_per_tick 782.9625
mozart/mz_570_2.mid
ms_per_tick 2695.12708333
mozart/mz_570_3.mid
ms_per_tick 854.408333333
mozart/mz_545_1.mid
ms_per_tick 929.022916667
mozart/mz_332_3.mid
ms_per_tick 827.86875
mozart/mz_330_1.mid
ms_per_tick 1862.89166667
mozart/mz_332_2.mid
ms_per_tick 1696.98541667
mozart/mz_545_2.mid
ms_per_tick 2234.9375
mozart/mz_330_2.mid
ms_per_tick 2688.17291667
mozart/mz_330_3.mid
ms_per_tick 1470.5875
mozart/mz_332_1.mid
ms_per_tick 862.425
mozart/mz_545_3.mid
ms_per_tick 1299.78125
mozart/mz_331_2.mid
ms_per_tick 1

Very low default error between emissions and hidden state. Possible challenge is that emissions are generated with near perfect tempo prediction.

In [7]:
for i in range(len(H_mat)):
    diffs = 0
    total = 0
    for j in range(len(H_mat[i])):
        if H_mat[i][j] != E_mat[i][j]:
            diffs += 1
        total += 1
    if total == 0:
        print filenames[i], 'has no notes'
    else:
        print filenames[i], 'Percent diffs: ', float(diffs) / total * 100

debussy/DEB_PASS.mid Percent diffs:  3.48977135981
debussy/debussy_cc_6.mid Percent diffs:  6.71755725191
debussy/debussy_cc_4.mid Percent diffs:  3.44506517691
debussy/deb_menu.mid Percent diffs:  4.21455938697
debussy/debussy_cc_3.mid Percent diffs:  2.0325203252
mozart/mz_570_1.mid Percent diffs:  0.802854594112
mozart/mz_570_2.mid Percent diffs:  1.49647887324
mozart/mz_570_3.mid Percent diffs:  0.388601036269
mozart/mz_545_1.mid Percent diffs:  0.564971751412
mozart/mz_332_3.mid Percent diffs:  1.76639434754
mozart/mz_330_1.mid Percent diffs:  0.393907563025
mozart/mz_332_2.mid Percent diffs:  1.12923462986
mozart/mz_545_2.mid Percent diffs:  6.36734693878
mozart/mz_330_2.mid Percent diffs:  2.64200792602
mozart/mz_330_3.mid Percent diffs:  1.37509549274
mozart/mz_332_1.mid Percent diffs:  0.29702970297
mozart/mz_545_3.mid Percent diffs:  1.54929577465
mozart/mz_331_2.mid Percent diffs:  1.03896103896
mozart/mz_333_1.mid Percent diffs:  1.07419435423
mozart/mz_331_3.mid Percent di

In [ ]:
E_mat[0]

In [8]:
model = HMM(possibleHMMBuckets(BUCKETS), nGramLength=3, laplace=20)

In [9]:
HMM_H = io_utils.generateHMMMatrix(H_mat)
HMM_E = io_utils.generateHMMMatrix(E_mat)

In [10]:
model.train(HMM_H, HMM_E)

[(None, None), (None, ((0.0625, 'd'), True)), (None, ((0.0625, 't'), True)), (None, ((0.0625,), True)), (None, ((0.125, 'd'), True)), (None, ((0.125, 't'), True)), (None, ((0.125,), True)), (None, ((0.25, 'd'), True)), (None, ((0.25, 't'), True)), (None, ((0.25,), True)), (None, ((0.5, 'd'), True)), (None, ((0.5,), True)), (None, ((1.0,), True)), (None, ((0.0625, 'd'), False)), (None, ((0.0625, 't'), False)), (None, ((0.0625,), False)), (None, ((0.125, 'd'), False)), (None, ((0.125, 't'), False)), (None, ((0.125,), False)), (None, ((0.25, 'd'), False)), (None, ((0.25, 't'), False)), (None, ((0.25,), False)), (None, ((0.5, 'd'), False)), (None, ((0.5,), False)), (None, ((1.0,), False))]
******************************************************


In [11]:
H_mat_complete, E_mat_complete, tempos_c, filenames_c, signatures_c = io_utils.generateTrainData(DIRS, BUCKETS, allowed_tempo_diff=1000)

[0.375, 0.16666666666666666, 0.25, 0.75, 0.3333333333333333, 0.5, 1.5, 0.6666666666666666, 1.0, 3.0, 2.0, 4.0]
debussy/DEB_CLAI.mid
ms_per_tick 1623.37708333
debussy/deb_prel.mid
ms_per_tick 1336.89791667
debussy/DEB_PASS.mid
ms_per_tick 719.177083333
debussy/debussy_cc_6.mid
ms_per_tick 1126.12708333
debussy/debussy_cc_4.mid
ms_per_tick 1011.98125
debussy/deb_menu.mid
ms_per_tick 1678.07708333
debussy/debussy_cc_1.mid
ms_per_tick 999.439583333
debussy/debussy_cc_2.mid
ms_per_tick 1201.11458333
debussy/debussy_cc_3.mid
ms_per_tick 920.675
mozart/mz_570_1.mid
ms_per_tick 782.9625
mozart/mz_570_2.mid
ms_per_tick 2695.12708333
mozart/mz_570_3.mid
ms_per_tick 854.408333333
mozart/mz_545_1.mid
ms_per_tick 929.022916667
mozart/mz_332_3.mid
ms_per_tick 827.86875
mozart/mz_330_1.mid
ms_per_tick 1862.89166667
mozart/mz_332_2.mid
ms_per_tick 1696.98541667
mozart/mz_545_2.mid
ms_per_tick 2234.9375
mozart/mz_330_2.mid
ms_per_tick 2688.17291667
mozart/mz_330_3.mid
ms_per_tick 1470.5875
mozart/mz_33

In [12]:
HMM_E_complete = io_utils.generateHMMMatrix(E_mat_complete)
# predictions = model.predict(HMM_E_complete)
two_predictions = model.predict(HMM_E_complete[:2])

dp dims: 576 x 487
S = 576
0/2
dp dims: 576 x 1036
S = 576


In [13]:
# smoothed_E_complete = io_utils.incorporatePrediction(H_mat_complete, predictions)
smoothed_E_complete = io_utils.incorporatePrediction(H_mat_complete[:2], two_predictions)

In [14]:
outputDiffs(H_mat_complete[:2], smoothed_E_complete, filenames_c[:2])

debussy/DEB_CLAI.mid Percent diffs:  100.0
debussy/deb_prel.mid Percent diffs:  99.1312741313


In [ ]:
outputDiffs(H_mat_complete, E_mat_complete, filenames_c)

In [15]:
for i in range(len(smoothed_E_complete)):
    timeSig, keySig = signatures_c[i]
    io_utils.eventListToMIDI(smoothed_E_complete[i], BUCKETS, 480, tempos_c[i], \
            filenames_c[i][:-4]+'_E_smoothed2.mid', output_dir=OUTPUT_DIR, \
            timeSig=timeSig, keySig=keySig)

midi.TimeSignatureEvent(tick=0, data=[9, 3, 12, 8])
0
midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[11, 227, 212]),
   midi.TimeSignatureEvent(tick=0, data=[9, 3, 12, 8]),
   midi.KeySignatureEvent(tick=0, data=[251, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[80, 36]),
   midi.NoteOnEvent(tick=0, channel=0, data=[77, 36]),
   midi.NoteOnEvent(tick=180, channel=0, data=[80, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[77, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[77, 33]),
   midi.NoteOnEvent(tick=0, channel=0, data=[73, 33]),
   midi.NoteOnEvent(tick=180, channel=0, data=[77, 0])])
midi.TimeSignatureEvent(tick=0, data=[4, 2, 24, 8])
0
midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[9, 202, 175]),
   midi.TimeSignatureEvent(tick=0, data=[4, 2, 24, 8]),
   midi.KeySignatureEvent(tick=0, data=[255, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[67, 83]),
   midi.NoteOnEvent(tick=0, channel=0, data=[70, 83]),
   midi.NoteOnEvent(tick=0, channel=0, data=[74, 83]),